In [8]:
import scipy
from IPython.display import Audio
from transformers import VitsTokenizer, VitsModel, set_seed

import pybangla
nrml = pybangla.Normalizer()

model_path = "intelsense/vits_mms_ft_openslr"
tokenizer = VitsTokenizer.from_pretrained(model_path)
model = VitsModel.from_pretrained(model_path).to("cuda").eval()

set_seed(456)  # make deterministic

In [9]:
def inference(raw_text, speaker_id=1):
    normalized_text = nrml.text_normalizer(raw_text)
    print(normalized_text)
    inputs = tokenizer(normalized_text, return_tensors="pt")
    outputs = model(inputs["input_ids"].to("cuda"), speaker_id=speaker_id)
    waveform = outputs.waveform[0].to("cpu").detach().numpy()
    return waveform

In [10]:
# get all speakers
model.config.num_speakers

16

In [13]:
texts = ["১৯৯৬ সালের ৬ সেপ্টেম্বর রণ ভ্রমণ পরিকল্পনা করছি ২০৩০ সালের ৬ সেপ্টেম্বর",
"উপসচিব পদে পদোন্নতির ক্ষেত্রে প্রশাসন ক্যাডারের কর্মকর্তাদের জন্য ৫০ শতাংশ ও অন্য ক্যাডারের জন্য ৫০ শতাংশ করা বিষয়ে জনপ্রশাসন সংস্কার কমিশনের সুপারিশ নিয়ে প্রশাসন ক্যাডারের কর্মকর্তাদের মধ্যে তীব্র আপত্তি উঠেছে।",
"ইজতেমা মাঠে হামলা ও সংঘর্ষে চারজন নিহত হয়েছেন জানিয়ে স্বরাষ্ট্র উপদেষ্টা লেফটেন্যান্ট জেনারেল (অব.) জাহাঙ্গীর আলম চৌধুরী বলেছেন, হত্যাকাণ্ডের সঙ্গে যারা জড়িত, তাদের ছাড় দেওয়ার কোনো অবকাশ নেই। ",
"অন্যদিকে শিক্ষা ও স্বাস্থ্যকে ক্যাডারে অন্তর্ভুক্ত না রেখে আলাদা কমিশনে রাখার যে সুপারিশ করা হচ্ছে, তা নিয়ে শিক্ষা ক্যাডারের কর্মকর্তারাও আপত্তি জানাচ্ছেন।",
"উদ্ভূত পরিস্থিতিতে আজ স্বরাষ্ট্র মন্ত্রণালয়ে দুই পক্ষের সঙ্গে আলাদাভাবে বৈঠক করেন স্বরাষ্ট্র উপদেষ্টা।",
"রংপুর অফিস, সম্পা ভিউ (নিচতলা), বাড়ি: ৫৬/১, রোড: ০১, আখতার সরণি, মুন্সিপাড়া, রংপুর, ফোন: ০১৭১১০৭০৬৭৪",
"বছরের শেষে ইন্টারনেট মুভি ডেটাবেজ (আইএমডিবি) ভারতের সবচেয়ে জনপ্রিয় ১০ তারকার তালিকা করেছে।",
"অন্যদিকে বাহুবলী তারকা ‘কাল্কি ২৮৯৮ এডি’ সিনেমা দিয়ে এ বছর আলোচনায় ছিলেন।",
"১৯৯১ সালে ব্রিটিশ অভিনেতা মাইকেল উইলিয়ামকে বিয়ে করেন অভিনেত্রী জুডি ডেঞ্চ।"]

for id, text in enumerate(texts):
    waveform = inference(text, speaker_id=0)
    scipy.io.wavfile.write(f"audio_samples/openslr_100_{id}.mp3", rate=model.config.sampling_rate, data=waveform)

# waveform = inference(texts[0], speaker_id=0)
# scipy.io.wavfile.write(f"audio_samples/openslr_100_0", rate=model.config.sampling_rate, data=waveform)
# Audio(waveform, rate=model.config.sampling_rate)

উনিশশো ছিয়ানব্বই সালের ছয় সেপ্টেম্বর রণ ভ্রমণ পরিকল্পনা করছি দুই হাজার ত্রিশ সালের ছয় সেপ্টেম্বর
উপসচিব পদে পদোন্নতির ক্ষেত্রে প্রশাসন ক্যাডারের কর্মকর্তাদের জন্য পঞ্চাশ শতাংশ ও অন্য ক্যাডারের জন্য পঞ্চাশ শতাংশ করা বিষয়ে জনপ্রশাসন সংস্কার কমিশনের সুপারিশ নিয়ে প্রশাসন ক্যাডারের কর্মকর্তাদের মধ্যে তীব্র আপত্তি উঠেছে।
ইজতেমা মাঠে হামলা ও সংঘর্ষে চারজন নিহত হয়েছেন জানিয়ে স্বরাষ্ট্র উপদেষ্টা লেফটেন্যান্ট জেনারেল অব . জাহাঙ্গীর আলম চৌধুরী বলেছেন, হত্যাকাণ্ডের সঙ্গে যারা জড়িত, তাদের ছাড় দেওয়ার কোনো অবকাশ নেই।
অন্যদিকে শিক্ষা ও স্বাস্থ্যকে ক্যাডারে অন্তর্ভুক্ত না রেখে আলাদা কমিশনে রাখার যে সুপারিশ করা হচ্ছে, তা নিয়ে শিক্ষা ক্যাডারের কর্মকর্তারাও আপত্তি জানাচ্ছেন।
উদ্ভূত পরিস্থিতিতে আজ স্বরাষ্ট্র মন্ত্রণালয়ে দুই পক্ষের সঙ্গে আলাদাভাবে বৈঠক করেন স্বরাষ্ট্র উপদেষ্টা।
রংপুর অফিস, সম্পা ভিউ নিচতলা বাড়ি ছাপ্পান্ন / এক রোড এক আখতার সরণি, মুন্সিপাড়া, রংপুর, ফোন জিরো ওয়ান সেভেন ডাবল ওয়ান জিরো সেভেন জিরো সিক্স সেভেন ফোর
বছরের শেষে ইন্টারনেট মুভি ডেটাবেজ আইএমডিবি ভারতের সবচেয়ে জনপ্রিয় দশ তারকার তালিকা 